In [1]:
%%capture
!pip install neuralprophet
!pip install prophet

In [2]:
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet, set_log_level, set_random_seed
from enum import auto
from prophet.serialize import model_to_json, model_from_json
import pickle

set_log_level("ERROR")
set_random_seed(888)

data = pd.read_csv('hist_data_all.csv', parse_dates=['Date'], index_col='Date')
# Build the dataset of pre-selected coins
dsets = {}
for t in list(data.Ticker.unique()):
  df = data.query('Ticker==@t').sort_index()
  df = df.reset_index()[['Date', 'Close', 'Gtrend', 'TYX','Volume']].rename(columns={'Date': 'ds', 'Close': 'y'})
  dsets[t] = df.drop_duplicates(subset='ds')
  dsets['BTC-USD']

  
splited_dsets = {}
for k, v in dsets.items():
    test_size = 365 if v.shape[0] >= 2000 else 120
    splited_dsets[k] = v.iloc[:-test_size, :], v.iloc[-test_size:, :]


def format_for_app(df, pivot_cols=False, pivot_id=[], var_names='Metric', value_names='Error'):
  if pivot_cols:
    id_vars = pivot_id if pivot_id else None
    df = pd.melt(df, value_vars=list(df.columns), id_vars=id_vars, var_name=var_names, value_name=value_names)
  df['Ticker'] = cc[:3]
  df['Model'] = 'Neural Prophet'
  df['Scope'] = '1 day ahead'
  return df

In [3]:
splited_dsets.keys()

dict_keys(['BTC-USD', 'ETH-USD', 'LTC-USD', 'NMC-USD', 'PPC-USD', 'FTC-USD'])

In [4]:
count = 0
for cc in splited_dsets.keys():
  print(cc)
  rf_var = [x for x in dsets[cc].columns.to_list() if x not in ('Gtrend', 'Volume', 'Close', 'y', 'ds')][0]
  df_train, df_test = splited_dsets[cc]
  m = NeuralProphet(
      trend_reg=2,
      yearly_seasonality=False,
      weekly_seasonality=True,
      daily_seasonality=False,
      n_lags = 1,
      n_forecasts = 1,
      # We let parameters to be set automatically by model selector
      #num_hidden_layers=4,
      #d_hidden=32,
      #learning_rate=0.003,
  )
  # We make it multivariate
  m.add_lagged_regressor('Gtrend')
  m.add_lagged_regressor(rf_var)
  m.add_lagged_regressor('Volume')
  # Training
  metrics_train = m.fit(df=df_train, freq="D")
  metrics_test = m.test(df=df_test)
  # Compute and save error metrics
  metrics_test = format_for_app(metrics_test, True)
  if count > 0:
    metrics_prev = pd.read_csv('nprophet_test_error.csv')
    metrics_test = pd.concat([metrics_prev, metrics_test])
  metrics_test.to_csv('nprophet_test_error.csv', index=False)

  # Make Predictions on test set and save
  test_size = df_test.shape[0]
  pred_npro = m.predict(dsets[cc].iloc[-(test_size + 1):, :]).iloc[-test_size:, :]
  test_df = pred_npro[['ds', 'y', 'yhat1']].rename(columns={'ds':'Date', 'y': 'Observed', 'yhat1': 'Predicted'})
  test_df = format_for_app(test_df, False)
  if count > 0:
    test_prev = pd.read_csv('nprophet_predictions.csv')
    test_df = pd.concat([test_prev, test_df])
  test_df.to_csv('nprophet_predictions.csv')

  # Feature contribution
  feat_list = pred_npro.columns.to_list()
  feat_list.remove('y')
  feat_list.remove('yhat1')
  fi_npro = pred_npro[feat_list].rename(columns={'ds': 'Date_dt'})
  fi_npro['Date'] = fi_npro['Date_dt'].dt.date.astype(str)
  fi_npro = format_for_app(fi_npro, True, pivot_id=['Date', 'Date_dt'], var_names='Feature', value_names='Contribution')
  if count > 0:
    fi_prev = pd.read_csv('nprophet_ft_importance.csv')
    fi_npro = pd.concat([fi_prev, fi_npro])
  fi_npro.to_csv('nprophet_ft_importance.csv')

  # Retrain Model on whole dataset
  m_prod = NeuralProphet(
      trend_reg=2,
      yearly_seasonality=False,
      weekly_seasonality=True,
      daily_seasonality=False,
      n_lags = 1,
      n_forecasts = 1,
      # We let parameters to be set automatically by model selector
      #num_hidden_layers=4,
      #d_hidden=32,
      #learning_rate=0.003,
  )
  # We make it multivariate
  m_prod.add_lagged_regressor('Gtrend')
  m_prod.add_lagged_regressor(rf_var)
  m_prod.add_lagged_regressor('Volume')
  # fit to complete dset
  m_prod.fit(dsets[cc], freq='D')
  # Save model to pkl
  pkl_path = cc[:3] + "_NeuralProphet.pkl"
  with open(pkl_path, "wb") as f:
      pickle.dump(m_prod, f)
  count += 1


BTC-USD


  0%|          | 0/135 [00:00<?, ?it/s]

  0%|          | 0/135 [00:00<?, ?it/s]

  0%|          | 0/136 [00:00<?, ?it/s]

  0%|          | 0/136 [00:00<?, ?it/s]

...

  0%|          | 0/135 [00:00<?, ?it/s]

FTC-USD


  0%|          | 0/135 [00:00<?, ?it/s]

  0%|          | 0/136 [00:00<?, ?it/s]

  0%|          | 0/136 [00:00<?, ?it/s]

In [ ]:
future_df = m.make_future_dataframe(df_train, 
                                    periods = df_test.shape[0], 
                                    n_historic_predictions=len(df_train))
preds_df = m.predict(future_df)
preds_df

,ds,y,yhat1,residual1,ar1,lagged_regressor_Gtrend1,lagged_regressor_TYX1,lagged_regressor_Volume1,trend,season_weekly
0,2014-09-17,0.031977,None,NaN,None,None,None,None,None,None
1,2014-09-18,0.038074,0.032304,-0.00577,0.029656,0.002382,0.000287,0.0,0.002078,0.000027
2,2014-09-19,0.027994,0.037902,0.009908,0.035713,0.002248,0.000287,0.0,0.002078,-0.000299
3,2014-09-20,0.040424,0.028728,-0.011696,0.025699,0.002126,0.00028,0.0,0.002078,0.000671
4,2014-09-21,0.028920,0.040599,0.011679,0.038048,0.002282,0.00028,0.0,0.002078,0.000037
...,...,...,...,...,...,...,...,...,...,...
2473,2021-06-25,0.021519,0.022064,0.000545,0.019939,0.00227,0.000137,0.0,0.002143,-0.000299
2474,2021-06-26,0.025035,0.022226,-0.002809,0.019266,0.002126,0.000146,0.0,0.002143,0.000671
2475,2021-06-27,0.026298,0.025085,-0.001213,0.022759,0.002126,0.000146,0.0,0.002143,0.000037
2476,2021-06-28,0.028597,0.026988,-0.001609,0.024014,0.002459,0.000146,0.0,0.002143,0.000351


### Load the Model

In [ ]:
with open('BTC_NeuralProphet.pkl', 'rb') as f:
    m = pickle.load(f)

### Make a Prediction

In [ ]:
future = m.make_future_dataframe(dsets['BTC-USD'].iloc[-2:, :])
m.predict(future)

,ds,y,yhat1,residual1,ar1,lagged_regressor_Gtrend1,lagged_regressor_TYX1,lagged_regressor_Volume1,trend,season_weekly
0,2022-06-28,20280.634766,None,NaN,None,None,None,None,None,None
1,2022-06-29,NaN,20172.841797,NaN,20045.232422,82.835831,-180.054962,28.076052,333.394409,41.460178


In [ ]:
dsets['BTC-USD']

,ds,y,Gtrend,TYX,Volume
0,2014-09-17,457.334015,47.000,3.364,21056800
1,2014-09-18,424.440002,47.000,3.359,34483200
2,2014-09-19,394.795990,53.000,3.299,37919700
3,2014-09-20,408.903992,46.000,3.299,36863600
4,2014-09-21,398.821014,48.000,3.299,26580100
...,...,...,...,...,...
2844,2022-06-24,21231.656250,47.000,3.257,24957784918
2845,2022-06-25,21502.337891,41.000,3.257,18372538715
2846,2022-06-26,21027.294922,40.000,3.257,18027170497
2847,2022-06-27,20735.478516,85.625,3.305,20965695707
